In [1]:
#!pip install rank-bm25
#from rank_bm25 import BM25Okapi
import pandas as pd
#import matplotlib.pyplot as plt
#import seaborn as sns
import os
from os import listdir
from os.path import isfile, join
import re
import numpy as np
from math import floor, ceil

import json
import gzip

from os import walk
from scipy.spatial import KDTree

!pip install geopy
!pip install phonenumbers
!pip install pycountry

import geopy.distance
import phonenumbers
import pycountry



In [2]:
path = r"../src/data/LocalBusiness/LocalBusiness_minimum3"
file_path = path + r"/geo_preprocessed"

files = os.listdir(file_path)


In [3]:
len(files)

11747

In [4]:
LBData = []

for lb in files:
    with gzip.open(file_path + '/' + lb, 'r') as dataFile:
        for line in dataFile:
            lineData = json.loads(line.decode('utf-8'))
            LBData.append(lineData)
df_all = pd.DataFrame(LBData)

In [5]:
len(df_all)

3053822

## Only get entries where dataframe has non-zero telephone numbers AND non-zero country codes

In [6]:
df_clean = df_all[~df_all["addresscountry"].isna()]
df_clean = df_clean[~df_clean["telephone"].isna()]
len(df_clean)

777204

In [7]:
lat = 'latitude'
lon = 'longitude'

df_clean = df_clean[df_clean[lon].notna()]
df_clean = df_clean[df_clean[lat].notna()]
len(df_clean)

445578

In [44]:
# Remove entries that are not numbers or cannot be convertred to one number (list etc.)
longitudes = df_clean[lon].to_numpy()
latitudes = df_clean[lat].to_numpy()
deleteList = []
i = 0

for value in longitudes:
    if ((isinstance(value, str) == False) & (isinstance(value, float) == False)):
        deleteList.append(i)
    i = i + 1

i = 0
for value in latitudes:
    if ((isinstance(value, str) == False) & (isinstance(value, float) == False)):
        deleteList.append(i)
    i = i + 1    

df_clean.drop(df_clean.index[deleteList], axis = 0, inplace = True)        

### Format longitude and latitude

In [48]:
longArray = df_clean[lon].to_numpy().astype(str)
longArray = np.char.replace(longArray, ',', '.')
longArray = np.char.replace(longArray, '--', '-')
df_clean[lon] = longArray
df_clean[lon] = pd.to_numeric(df_clean[lon], errors='coerce')

latArray = df_clean[lat].to_numpy().astype(str)
latArray = np.char.replace(latArray, ',', '.')
latArray = np.char.replace(latArray, '--', '-')
df_clean[lat] = latArray
df_clean[lat] = pd.to_numeric(df_clean[lat], errors='coerce')

In [51]:
# Remove the entries that were set to NaN because of other errors
df_clean = df_clean[df_clean[lon].notna()]
df_clean = df_clean[df_clean[lat].notna()]

In [52]:
pd.options.display.max_columns = 100
df_clean[:3]

,row_id,name,address,description,pricerange,founder,owns,page_url,addressregion,streetaddress,addresscountry,telephone_address,addresslocality,postalcode,telephone,geo,sameas,latitude,longitude,url,aggregaterating,image,openinghours,review,email,contactpoint,reviews,employees,faxnumber,about,citystatezip,openinghoursspecification,areaserved,location,paymentaccepted,currenciesaccepted,legalname,photos,photo,logo,alternatename,brand,department,makesoffer,map,award,hasmap,isicv4,foundingdate,parentorganization,...,productid,addressstate,specialopeninghoursspecification,suborganization,awards,founders,dealerzipcode,maplink,numberofitems,productsupported,foundinglocation,speakable,tourbookingpage,phone,itemprop=,"span>,",<span,participatesinmedicare,header-title-text,text-muted,containedin,primaryimageofpage,availablelanguage,userinteractioncount,acceptedpaymentmethod,hasmerchantreturnpolicy,postaladdress,address4,acceptsreservations,haspos,availableservice,position,type,virtualcategory,audience,restaurant,golfcourse,addressname,receivinghours,namecompany,provider,hascourseinstance,serialnumber,phonenumber,\u201daddresslocality\u201d,text,target,pickupinfo,\u201daddressregion\u201d,isrelatedto
51,1,Galga Studio,"{'addresslocality': 'Madrid', 'addresscountry'...",NaN,NaN,NaN,NaN,https://es.litmind.com/galga_studio,Madrid,calle Lorenzo Gonzalez 4,ES,NaN,Madrid,28017,+34,"{'latitude': '40.4315119', 'longitude': '-3.64...","{'telephone': '+34', 'name': 'Galga Studio'}",40.431512,-3.649591,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
52,2,Salvador Model Agency,"{'postalcode': '28001', 'addresslocality': 'Ma...",NaN,NaN,NaN,NaN,https://es.litmind.com/salvadormodelagency,Madrid,"General Pardiñas, 34. 1º7ª",ES,NaN,Madrid,28001,+34 914310707,"{'longitude': '-3.6835849285126', 'latitude': ...","{'name': 'Salvador Model Agency', 'telephone':...",40.423894,-3.683585,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
58,10,UNIC AZAFATAS S.L,"{'postalcode': '28250', 'addresscountry': 'ES'...",NaN,NaN,NaN,NaN,https://es.litmind.com/456459,Madrid,C/ TORRENCINA 18,ES,NaN,TORRELODONES,28250,+34 918599376,"{'longitude': '-3.8939949', 'latitude': '40.58...","{'name': 'UNIC AZAFATAS S.L', 'telephone': '+3...",40.580829,-3.893995,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Second preprocessing step

### Remove non-digits from telephone numbers

In [53]:
def remove_non_digits(string):
    cleaned = re.sub('[^0-9]','', string)
    return cleaned

In [54]:
df_clean['telephone_'] = df_clean['telephone'].astype('str').apply(remove_non_digits)

### Extract country codes to ISO-2 format using ``pycountry``

In [55]:
input_countries = df_clean["addresscountry"]

countries = {}
for country in pycountry.countries:
    countries[country.name] = country.alpha_2

countries


{'Aruba': 'AW',
 'Afghanistan': 'AF',
 'Angola': 'AO',
 'Anguilla': 'AI',
 'Åland Islands': 'AX',
 'Albania': 'AL',
 'Andorra': 'AD',
 'United Arab Emirates': 'AE',
 'Argentina': 'AR',
 'Armenia': 'AM',
 'American Samoa': 'AS',
 'Antarctica': 'AQ',
 'French Southern Territories': 'TF',
 'Antigua and Barbuda': 'AG',
 'Australia': 'AU',
 'Austria': 'AT',
 'Azerbaijan': 'AZ',
 'Burundi': 'BI',
 'Belgium': 'BE',
 'Benin': 'BJ',
 'Bonaire, Sint Eustatius and Saba': 'BQ',
 'Burkina Faso': 'BF',
 'Bangladesh': 'BD',
 'Bulgaria': 'BG',
 'Bahrain': 'BH',
 'Bahamas': 'BS',
 'Bosnia and Herzegovina': 'BA',
 'Saint Barthélemy': 'BL',
 'Belarus': 'BY',
 'Belize': 'BZ',
 'Bermuda': 'BM',
 'Bolivia, Plurinational State of': 'BO',
 'Brazil': 'BR',
 'Barbados': 'BB',
 'Brunei Darussalam': 'BN',
 'Bhutan': 'BT',
 'Bouvet Island': 'BV',
 'Botswana': 'BW',
 'Central African Republic': 'CF',
 'Canada': 'CA',
 'Cocos (Keeling) Islands': 'CC',
 'Switzerland': 'CH',
 'Chile': 'CL',
 'China': 'CN',
 "Côte d'Iv

In [56]:
for key, value in countries.items():
    df_clean["addresscountry"] = df_clean["addresscountry"].str.replace(key, value)

<ipython-input-56-6f0b15e99d94>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df_clean["addresscountry"] = df_clean["addresscountry"].str.replace(key, value)


In [57]:
df_clean["addresscountry"].unique()

array(['ES', 'AE', 'NL', 'PT', 'AT', 'DE', 'GR', 'QA', 'GB', 'IE', 'IS',
       'FI', 'MT', 'NO', 'CH', 'SE', 'AU', 'Brasil', 'UNITED STATES',
       'USA', 'CA', 'DK', 'IN', 'US', 'Oroszország', 'רוסיה', 'Russland',
       'Россия', 'Rusia', 'روسيا', 'Rússia', 'Russia', 'Rusland',
       'Venäjä', 'Русия', 'Ruska', 'Nga', 'Rusya', 'Росія', 'روسيه',
       'Russie', 'ロシア', 'Ryssland', 'Rosja', '俄国', '러시아', 'Ρωσία', 'BR',
       'MX', 'Caribbean Sea', 'ID', 'BR - South America', 'SG', 'BD',
       'NZ', 'ZA', 'TH', 'İspanya', 'Şili', 'Arjantin', 'Brezilya',
       'Fransa', 'Almanya', 'Güney Afrika', 'Japonya', 'Portekiz',
       'Meksika', 'Kolombiya', 'Avustralya', 'Türkiye', 'Polonya', 'PE',
       'İtalya', 'Singapur', 'Hollanda', 'Hindistan', 'Endonezya',
       'Vietnam', 'Güney Kore', 'Malezya', 'Birleşik Krallık',
       'Rusya Federasyonu', 'Kanada', 'Tayvan',
       'Amerika Birleşik Devletleri', 'Mısır',
       'Birleşik Arap Emirlikleri', 'Azerbaycan', 'Belçika',
       'HK 

## Manually normalize countries which do not exist by default

In [58]:
country_dictionary = {"UNITED STATES": "US", "USA":"US", "UNITED KINGDOM": "GB", "UK": "GB", "CANADA": "CA", "AUSTRALIA": "AU", "UNITED ARAB EMIRATES": "AE" , "INDIA" : "IN", "NEW ZEALAND": "NZ"}
for key, value in country_dictionary.items():
    df_clean["addresscountry"] = df_clean["addresscountry"].str.replace(key, value)

In [59]:
df_clean["addresscountry"] = df_clean["addresscountry"].str.upper()

In [60]:
df_clean["addresscountry"].unique()

array(['ES', 'AE', 'NL', 'PT', 'AT', 'DE', 'GR', 'QA', 'GB', 'IE', 'IS',
       'FI', 'MT', 'NO', 'CH', 'SE', 'AU', 'BRASIL', 'US', 'CA', 'DK',
       'IN', 'OROSZORSZÁG', 'רוסיה', 'RUSSLAND', 'РОССИЯ', 'RUSIA',
       'روسيا', 'RÚSSIA', 'RUSSIA', 'RUSLAND', 'VENÄJÄ', 'РУСИЯ', 'RUSKA',
       'NGA', 'RUSYA', 'РОСІЯ', 'روسيه', 'RUSSIE', 'ロシア', 'RYSSLAND',
       'ROSJA', '俄国', '러시아', 'ΡΩΣΊΑ', 'BR', 'MX', 'CARIBBEAN SEA', 'ID',
       'BR - SOUTH AMERICA', 'SG', 'BD', 'NZ', 'ZA', 'TH', 'İSPANYA',
       'ŞILI', 'ARJANTIN', 'BREZILYA', 'FRANSA', 'ALMANYA',
       'GÜNEY AFRIKA', 'JAPONYA', 'PORTEKIZ', 'MEKSIKA', 'KOLOMBIYA',
       'AVUSTRALYA', 'TÜRKIYE', 'POLONYA', 'PE', 'İTALYA', 'SINGAPUR',
       'HOLLANDA', 'HINDISTAN', 'ENDONEZYA', 'VIETNAM', 'GÜNEY KORE',
       'MALEZYA', 'BIRLEŞIK KRALLIK', 'RUSYA FEDERASYONU', 'KANADA',
       'TAYVAN', 'AMERIKA BIRLEŞIK DEVLETLERI', 'MISIR',
       'BIRLEŞIK ARAP EMIRLIKLERI', 'AZERBAYCAN', 'BELÇIKA',
       'HK SAR – ÇIN', 'SUUDI ARABISTAN', 

## Remove non-covered countries

In [61]:
df_clean.reset_index(inplace=True)

In [62]:
liste = []
for i, row in enumerate(df_clean["addresscountry"]):
    if len(row) > 2:
        liste.append(i)
        
df_clean = df_clean.drop(liste)        

In [63]:
df_clean["addresscountry"].unique()

array(['ES', 'AE', 'NL', 'PT', 'AT', 'DE', 'GR', 'QA', 'GB', 'IE', 'IS',
       'FI', 'MT', 'NO', 'CH', 'SE', 'AU', 'US', 'CA', 'DK', 'IN', '俄国',
       'BR', 'MX', 'ID', 'SG', 'BD', 'NZ', 'ZA', 'TH', 'PE', 'PK', 'BZ',
       'SK', 'BE', 'PL', 'IT', 'MY', 'FR', 'GN', 'TR', 'SA', 'EG', 'CU',
       'UA', 'CY', 'PH', 'VN', 'KY', 'CN', 'JP', 'HK', 'AR', 'LI', 'DZ',
       'GY', 'BA', 'BW', 'MM', 'AZ', 'MZ', 'KW', 'TT', 'MA', 'KH', 'LY',
       'LK', 'GE', 'JM', 'PY', 'EE', 'AF', 'JE', 'GI', 'GG', 'ME', 'MV',
       'CR', 'AM', 'BH', 'HU', 'RW', 'JO', 'SV', 'BB', 'GT', 'CO', 'BS',
       'OM', 'GA', 'GU', 'MU', 'BG', 'KE', 'LA', 'SN', 'TD', 'LU', 'LC',
       'MG', 'IM', '오만', 'NI', 'BM', 'AW', 'UG', 'HR', 'AL', 'LV', 'AO',
       'UY', 'UZ', 'MP', 'MK', '맨섬', '인도', '건지', 'GL', 'MN', 'PA', '중국',
       '저지', '米国', '몽골', 'SI', '케냐', 'TN', 'RS', '차드', '가봉', '독일', '英国',
       '몰타', '괌', 'SY', 'KN', 'FL', 'RU', 'MD', 'BO', 'LT', 'BF', 'BJ',
       'ET', 'IL', 'TZ', 'PR', 'TW', 'NP', 'KG', 'CZ

## Drop empty phonenumbers and too lenghty phone numbers

In [64]:
df_clean = df_clean[df_clean["telephone_"] != "" ]

In [65]:
liste = []
df_clean.reset_index(inplace=True)
for row_index in df_clean.index:
    if len(df_clean.iloc[row_index]["telephone_"])>18:
        liste.append(row_index)

In [66]:
df_clean.drop(labels = liste, inplace = True)

In [67]:
df_clean = df_clean.drop(columns = "index")
df_clean.head()

,level_0,row_id,name,address,description,pricerange,founder,owns,page_url,addressregion,streetaddress,addresscountry,telephone_address,addresslocality,postalcode,telephone,geo,sameas,latitude,longitude,url,aggregaterating,image,openinghours,review,email,contactpoint,reviews,employees,faxnumber,about,citystatezip,openinghoursspecification,areaserved,location,paymentaccepted,currenciesaccepted,legalname,photos,photo,logo,alternatename,brand,department,makesoffer,map,award,hasmap,isicv4,foundingdate,...,addressstate,specialopeninghoursspecification,suborganization,awards,founders,dealerzipcode,maplink,numberofitems,productsupported,foundinglocation,speakable,tourbookingpage,phone,itemprop=,"span>,",<span,participatesinmedicare,header-title-text,text-muted,containedin,primaryimageofpage,availablelanguage,userinteractioncount,acceptedpaymentmethod,hasmerchantreturnpolicy,postaladdress,address4,acceptsreservations,haspos,availableservice,position,type,virtualcategory,audience,restaurant,golfcourse,addressname,receivinghours,namecompany,provider,hascourseinstance,serialnumber,phonenumber,\u201daddresslocality\u201d,text,target,pickupinfo,\u201daddressregion\u201d,isrelatedto,telephone_
0,0,1,Galga Studio,"{'addresslocality': 'Madrid', 'addresscountry'...",NaN,NaN,NaN,NaN,https://es.litmind.com/galga_studio,Madrid,calle Lorenzo Gonzalez 4,ES,NaN,Madrid,28017,+34,"{'latitude': '40.4315119', 'longitude': '-3.64...","{'telephone': '+34', 'name': 'Galga Studio'}",40.431512,-3.649591,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,34
1,1,2,Salvador Model Agency,"{'postalcode': '28001', 'addresslocality': 'Ma...",NaN,NaN,NaN,NaN,https://es.litmind.com/salvadormodelagency,Madrid,"General Pardiñas, 34. 1º7ª",ES,NaN,Madrid,28001,+34 914310707,"{'longitude': '-3.6835849285126', 'latitude': ...","{'name': 'Salvador Model Agency', 'telephone':...",40.423894,-3.683585,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,34914310707
2,2,10,UNIC AZAFATAS S.L,"{'postalcode': '28250', 'addresscountry': 'ES'...",NaN,NaN,NaN,NaN,https://es.litmind.com/456459,Madrid,C/ TORRENCINA 18,ES,NaN,TORRELODONES,28250,+34 918599376,"{'longitude': '-3.8939949', 'latitude': '40.58...","{'name': 'UNIC AZAFATAS S.L', 'telephone': '+3...",40.580829,-3.893995,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,34918599376
3,3,16,Coconut,"{'streetaddress': 'Calle Zabaleta, 10', 'addre...",NaN,NaN,NaN,NaN,https://es.litmind.com/coconutmadrid/info,Madrid,"Calle Zabaleta, 10",ES,NaN,Madrid,28002,+34 913776214,"{'latitude': '40.4413414', 'longitude': '-3.67...","{'name': 'Coconut', 'telephone': '+34 913776214'}",40.441341,-3.674406,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,34913776214
4,4,18,SDA MODEL'S INTERNATIONAL AGENCY,"{'postalcode': '28001', 'addresscountry': 'ES'...",NaN,NaN,NaN,NaN,https://es.litmind.com/sdamodelsintagency,Madrid,ayala madrid n7 bajos,ES,NaN,barcelona,28001,+34 640847492,"{'longitude':

## Define normalizer for telephone package phonenumbers

In [68]:
def normalizer(entity):
    number = entity["telephone_"]
    address_country = entity["addresscountry"]
    phone_number = phonenumbers.parse(number, address_country)
    return phone_number

## Finally normalizing phone numbers in E.164 format

### Ignore those which can not be identified and replace as ``unknown``

In [69]:
df_clean.reset_index(inplace=True)
phone_objects =[]
#index = []
for row_index in df_clean.index:
    try:
        phone_object =  normalizer(df_clean.iloc[row_index])
        #index.append(row_index)
        phone_objects.append(phone_object)
    except:
        phone_objects.append("unknown")
    
    

In [70]:
len(phone_objects)

406543

In [71]:
df_clean["phone_object"] = pd.Series(phone_objects)

In [72]:
df_clean = df_clean.drop(columns = "index")
df_clean.head()

,level_0,row_id,name,address,description,pricerange,founder,owns,page_url,addressregion,streetaddress,addresscountry,telephone_address,addresslocality,postalcode,telephone,geo,sameas,latitude,longitude,url,aggregaterating,image,openinghours,review,email,contactpoint,reviews,employees,faxnumber,about,citystatezip,openinghoursspecification,areaserved,location,paymentaccepted,currenciesaccepted,legalname,photos,photo,logo,alternatename,brand,department,makesoffer,map,award,hasmap,isicv4,foundingdate,...,specialopeninghoursspecification,suborganization,awards,founders,dealerzipcode,maplink,numberofitems,productsupported,foundinglocation,speakable,tourbookingpage,phone,itemprop=,"span>,",<span,participatesinmedicare,header-title-text,text-muted,containedin,primaryimageofpage,availablelanguage,userinteractioncount,acceptedpaymentmethod,hasmerchantreturnpolicy,postaladdress,address4,acceptsreservations,haspos,availableservice,position,type,virtualcategory,audience,restaurant,golfcourse,addressname,receivinghours,namecompany,provider,hascourseinstance,serialnumber,phonenumber,\u201daddresslocality\u201d,text,target,pickupinfo,\u201daddressregion\u201d,isrelatedto,telephone_,phone_object
0,0,1,Galga Studio,"{'addresslocality': 'Madrid', 'addresscountry'...",NaN,NaN,NaN,NaN,https://es.litmind.com/galga_studio,Madrid,calle Lorenzo Gonzalez 4,ES,NaN,Madrid,28017,+34,"{'latitude': '40.4315119', 'longitude': '-3.64...","{'telephone': '+34', 'name': 'Galga Studio'}",40.431512,-3.649591,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,34,Country Code: 34 National Number: 34
1,1,2,Salvador Model Agency,"{'postalcode': '28001', 'addresslocality': 'Ma...",NaN,NaN,NaN,NaN,https://es.litmind.com/salvadormodelagency,Madrid,"General Pardiñas, 34. 1º7ª",ES,NaN,Madrid,28001,+34 914310707,"{'longitude': '-3.6835849285126', 'latitude': ...","{'name': 'Salvador Model Agency', 'telephone':...",40.423894,-3.683585,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,34914310707,Country Code: 34 National Number: 914310707
2,2,10,UNIC AZAFATAS S.L,"{'postalcode': '28250', 'addresscountry': 'ES'...",NaN,NaN,NaN,NaN,https://es.litmind.com/456459,Madrid,C/ TORRENCINA 18,ES,NaN,TORRELODONES,28250,+34 918599376,"{'longitude': '-3.8939949', 'latitude': '40.58...","{'name': 'UNIC AZAFATAS S.L', 'telephone': '+3...",40.580829,-3.893995,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,34918599376,Country Code: 34 National Number: 918599376
3,3,16,Coconut,"{'streetaddress': 'Calle Zabaleta, 10', 'addre...",NaN,NaN,NaN,NaN,https://es.litmind.com/coconutmadrid/info,Madrid,"Calle Zabaleta, 10",ES,NaN,Madrid,28002,+34 913776214,"{'latitude': '40.4413414', 'longitude': '-3.67...","{'name': 'Coconut', 'telephone': '+34 913776214'}",40.441341,-3.674406,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,34913776214,Country Code: 34 National Number: 913776214
4,4,18,SDA MODEL'S INTERNATIONAL AGENCY,"{'postalcode': '28001', 'addresscoun

In [73]:
unknown_rows = df_clean[df_clean["phone_object"] == "unknown"].index

In [74]:
df_clean = df_clean.drop(unknown_rows)

In [75]:
len(df_clean)

406149

## Check whether phonenumbers are valid

In [76]:
df_valid_numbers = df_clean[df_clean["phone_object"].apply(phonenumbers.is_valid_number)]

In [77]:
len(df_valid_numbers)

378678

## Finally dataset is reduced to roughly 78.k entries
## startet with roughly 500.k entries this reduces size by 80%

## Next step: Format every telephone number into unique E.164 format

In [78]:
#phonenumbers.format_number(df_valid_numbers["phone_object"][0], phonenumbers.PhoneNumberFormat.E164)

In [79]:
df_valid_numbers["E.164 format"] = df_valid_numbers["phone_object"].apply(lambda object: phonenumbers.format_number(object, phonenumbers.PhoneNumberFormat.E164))

<ipython-input-79-8dc4dbf6ff4f>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_valid_numbers["E.164 format"] = df_valid_numbers["phone_object"].apply(lambda object: phonenumbers.format_number(object, phonenumbers.PhoneNumberFormat.E164))


In [80]:
len(df_valid_numbers)

378678

In [81]:
df_valid_numbers.head()

,level_0,row_id,name,address,description,pricerange,founder,owns,page_url,addressregion,streetaddress,addresscountry,telephone_address,addresslocality,postalcode,telephone,geo,sameas,latitude,longitude,url,aggregaterating,image,openinghours,review,email,contactpoint,reviews,employees,faxnumber,about,citystatezip,openinghoursspecification,areaserved,location,paymentaccepted,currenciesaccepted,legalname,photos,photo,logo,alternatename,brand,department,makesoffer,map,award,hasmap,isicv4,foundingdate,...,suborganization,awards,founders,dealerzipcode,maplink,numberofitems,productsupported,foundinglocation,speakable,tourbookingpage,phone,itemprop=,"span>,",<span,participatesinmedicare,header-title-text,text-muted,containedin,primaryimageofpage,availablelanguage,userinteractioncount,acceptedpaymentmethod,hasmerchantreturnpolicy,postaladdress,address4,acceptsreservations,haspos,availableservice,position,type,virtualcategory,audience,restaurant,golfcourse,addressname,receivinghours,namecompany,provider,hascourseinstance,serialnumber,phonenumber,\u201daddresslocality\u201d,text,target,pickupinfo,\u201daddressregion\u201d,isrelatedto,telephone_,phone_object,E.164 format
1,1,2,Salvador Model Agency,"{'postalcode': '28001', 'addresslocality': 'Ma...",NaN,NaN,NaN,NaN,https://es.litmind.com/salvadormodelagency,Madrid,"General Pardiñas, 34. 1º7ª",ES,NaN,Madrid,28001,+34 914310707,"{'longitude': '-3.6835849285126', 'latitude': ...","{'name': 'Salvador Model Agency', 'telephone':...",40.423894,-3.683585,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,34914310707,Country Code: 34 National Number: 914310707,+34914310707
2,2,10,UNIC AZAFATAS S.L,"{'postalcode': '28250', 'addresscountry': 'ES'...",NaN,NaN,NaN,NaN,https://es.litmind.com/456459,Madrid,C/ TORRENCINA 18,ES,NaN,TORRELODONES,28250,+34 918599376,"{'longitude': '-3.8939949', 'latitude': '40.58...","{'name': 'UNIC AZAFATAS S.L', 'telephone': '+3...",40.580829,-3.893995,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,34918599376,Country Code: 34 National Number: 918599376,+34918599376
3,3,16,Coconut,"{'streetaddress': 'Calle Zabaleta, 10', 'addre...",NaN,NaN,NaN,NaN,https://es.litmind.com/coconutmadrid/info,Madrid,"Calle Zabaleta, 10",ES,NaN,Madrid,28002,+34 913776214,"{'latitude': '40.4413414', 'longitude': '-3.67...","{'name': 'Coconut', 'telephone': '+34 913776214'}",40.441341,-3.674406,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,34913776214,Country Code: 34 National Number: 913776214,+34913776214
4,4,18,SDA MODEL'S INTERNATIONAL AGENCY,"{'postalcode': '28001', 'addresscountry': 'ES'...",NaN,NaN,NaN,NaN,https://es.litmind.com/sdamodelsintagency,Madrid,ayala madrid n7 bajos,ES,NaN,barcelona,28001,+34 640847492,"{'longitude': '-3.6825650000001', 'latitude': ...","{'name': 'SDA MODEL'S INTERNATIONAL AGENCY', '...",40.427645,-3.682565,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,34640847492,Country Code: 34 National Number: 640847492,+34640847492
5,5

## After formatting phone numbers into unified format we can group by phone numbers to identify clusters

In [82]:
df_valid_numbers["E.164 format"].value_counts().sort_values().tail(40)

+18004633339         256
+13366178823         258
+611300244743        275
+61499000201         287
+443301289355        299
+18174381209         304
+33974593330         305
+61370180758         312
+17137824911         316
+18005300431         330
+61432607539         337
+61410160096         341
+12693443600         342
+351915225592        353
+18004264650         370
+12029081313         373
+3224630970          380
+61359753874         384
+13058858836         386
+443300886116        396
+611300769302        408
+18003271040         411
+48570600559         416
+61418515434         469
+49492151839839      471
+908505502060        494
+421905383904        516
+16179368365         757
+611300724607        824
+18448592146        1032
+18552674854        1036
+33149461720        1163
+18664770844        1278
+18775784138        1347
+443458121007       1701
+17136954939        2053
+18664773715        2172
+611300032861       2209
+611300695278       2970
+18884082399       14917


In [83]:
pd.set_option('display.max_columns', 500)
df_valid_numbers[df_valid_numbers["E.164 format"] == "+918047026626"][:5]

,level_0,row_id,name,address,description,pricerange,founder,owns,page_url,addressregion,streetaddress,addresscountry,telephone_address,addresslocality,postalcode,telephone,geo,sameas,latitude,longitude,url,aggregaterating,image,openinghours,review,email,contactpoint,reviews,employees,faxnumber,about,citystatezip,openinghoursspecification,areaserved,location,paymentaccepted,currenciesaccepted,legalname,photos,photo,logo,alternatename,brand,department,makesoffer,map,award,hasmap,isicv4,foundingdate,parentorganization,additionalproperty,city,state,county,employeeid,firstname,fullname,licensedata,nmlsid,officephone,suffdataforcta,numberofemployees,contactoption,ratingvalue,ratingcount,bestrating,worstrating,\u201caddresslocality\u201d,\u201cpostalcode\u201d,hasoffercatalog,itemreviewed,fax,postcode,startdate,enddate,amenityfeature,articlebody,naics,additionaltype,employee,whatsapp,datemodified,offers,category,community-number,knowslanguage,headline,author,datepublished,publisher,keywords,contacttype,postofficeboxnumber,vatid,member,additional_phone,breadcrumb,memberof,mainentityofpage,mainentity,interactioncount,slogan,elevation,distance,mobilebusiness,locationmobile,disambiguatingdescription,servicetype,potentialaction,servicearea,branchof,country,maximumattendeecapacity,knowsabout,containedinplace,id,isaccessibleforfree,branchcode,issimilarto,dateupdated,jobtitle,publicaccess,taxid,duns,hasmenu,identifier,leicode,servescuisine,mail,adresse,nom,validation%20de%20l'avis,gu00e9olocalisation,tu00e9lu00e9phone,inlanguage,reviewcount,reviewbody,caption,servicephone,maddress,smokingallowed,mobile,datecreated,alternativeheadline,events,contactpoints,price,itemlistelement,pricecurrency,availability,businessfunction,tollfree,reviewrating,clearfix,numberofrooms,eht,thumbnailurl,streetaddress2,website,blogpost,ispartof,event,webaddress,opens,closes,sku,\u201caddressregion\u201d,interactionstatistic,circle,title,listingemail,height,width,containsplace,maincontentofpage,hascredential,comment,georadius,place_location,place_id,place_name,dateposted,occupationalcategory,end_date,post_id,start_date,mpn,zipcode,productid,addressstate,specialopeninghoursspecification,suborganization,awards,founders,dealerzipcode,maplink,numberofitems,productsupported,foundinglocation,speakable,tourbookingpage,phone,itemprop=,"span>,",<span,participatesinmedicare,header-title-text,text-muted,containedin,primaryimageofpage,availablelanguage,userinteractioncount,acceptedpaymentmethod,hasmerchantreturnpolicy,postaladdress,address4,acceptsreservations,haspos,availableservice,position,type,virtualcategory,audience,restaurant,golfcourse,addressname,receivinghours,namecompany,provider,hascourseinstance,serialnumber,phonenumber,\u201daddresslocality\u201d,text,target,pickupinfo,\u201daddressregion\u201d,isrelatedto,telephone_,phone_object,E.164 format


## As one can see from the geo locations this is a *successful* match!

In [84]:
df_valid_numbers[df_valid_numbers["E.164 format"] == "+566006373737"][:5]

,level_0,row_id,name,address,description,pricerange,founder,owns,page_url,addressregion,streetaddress,addresscountry,telephone_address,addresslocality,postalcode,telephone,geo,sameas,latitude,longitude,url,aggregaterating,image,openinghours,review,email,contactpoint,reviews,employees,faxnumber,about,citystatezip,openinghoursspecification,areaserved,location,paymentaccepted,currenciesaccepted,legalname,photos,photo,logo,alternatename,brand,department,makesoffer,map,award,hasmap,isicv4,foundingdate,parentorganization,additionalproperty,city,state,county,employeeid,firstname,fullname,licensedata,nmlsid,officephone,suffdataforcta,numberofemployees,contactoption,ratingvalue,ratingcount,bestrating,worstrating,\u201caddresslocality\u201d,\u201cpostalcode\u201d,hasoffercatalog,itemreviewed,fax,postcode,startdate,enddate,amenityfeature,articlebody,naics,additionaltype,employee,whatsapp,datemodified,offers,category,community-number,knowslanguage,headline,author,datepublished,publisher,keywords,contacttype,postofficeboxnumber,vatid,member,additional_phone,breadcrumb,memberof,mainentityofpage,mainentity,interactioncount,slogan,elevation,distance,mobilebusiness,locationmobile,disambiguatingdescription,servicetype,potentialaction,servicearea,branchof,country,maximumattendeecapacity,knowsabout,containedinplace,id,isaccessibleforfree,branchcode,issimilarto,dateupdated,jobtitle,publicaccess,taxid,duns,hasmenu,identifier,leicode,servescuisine,mail,adresse,nom,validation%20de%20l'avis,gu00e9olocalisation,tu00e9lu00e9phone,inlanguage,reviewcount,reviewbody,caption,servicephone,maddress,smokingallowed,mobile,datecreated,alternativeheadline,events,contactpoints,price,itemlistelement,pricecurrency,availability,businessfunction,tollfree,reviewrating,clearfix,numberofrooms,eht,thumbnailurl,streetaddress2,website,blogpost,ispartof,event,webaddress,opens,closes,sku,\u201caddressregion\u201d,interactionstatistic,circle,title,listingemail,height,width,containsplace,maincontentofpage,hascredential,comment,georadius,place_location,place_id,place_name,dateposted,occupationalcategory,end_date,post_id,start_date,mpn,zipcode,productid,addressstate,specialopeninghoursspecification,suborganization,awards,founders,dealerzipcode,maplink,numberofitems,productsupported,foundinglocation,speakable,tourbookingpage,phone,itemprop=,"span>,",<span,participatesinmedicare,header-title-text,text-muted,containedin,primaryimageofpage,availablelanguage,userinteractioncount,acceptedpaymentmethod,hasmerchantreturnpolicy,postaladdress,address4,acceptsreservations,haspos,availableservice,position,type,virtualcategory,audience,restaurant,golfcourse,addressname,receivinghours,namecompany,provider,hascourseinstance,serialnumber,phonenumber,\u201daddresslocality\u201d,text,target,pickupinfo,\u201daddressregion\u201d,isrelatedto,telephone_,phone_object,E.164 format


## Adding the matching telephone numbers in a new column

In [85]:
def createKDTree(tupleArray):
    tree = KDTree(tupleArray)
    return tree

# Return all values that are in a specific proximity
def queryTree(tree, point, r = 0):    
    point = [float(i) for i in point]
    idx = tree.query_ball_point(point, r)
    return idx

df_valid_numbers['telephoneNorm'] = df_valid_numbers['E.164 format'].str.replace('+','').astype(np.int64)
df_valid_numbers.reset_index(drop=True, inplace=True)
df_valid_numbers['indexValue'] = df_valid_numbers.index
telephoneArray = df_valid_numbers['telephoneNorm'].to_numpy().astype('int64')
fillArray = np.full(len(telephoneArray), 1)
tupleArray = np.array((telephoneArray, fillArray)).T.astype('int64')


# create new column with all matching points
tree = createKDTree(tupleArray)
idx = queryTree(tree, tupleArray[0])

# Search for the closest neighbour in all of the points
df_valid_numbers['MatchingNumbers'] = df_valid_numbers.apply(lambda row: queryTree(tree,[row['telephoneNorm'], 1]), axis=1) 

<ipython-input-85-623d5e3ddf8f>:11: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  df_valid_numbers['telephoneNorm'] = df_valid_numbers['E.164 format'].str.replace('+','').astype(np.int64)
<ipython-input-85-623d5e3ddf8f>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_valid_numbers['telephoneNorm'] = df_valid_numbers['E.164 format'].str.replace('+','').astype(np.int64)
<ipython-input-85-623d5e3ddf8f>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: 

In [86]:
# filter out the values which only have one value
data = df_valid_numbers[df_valid_numbers['MatchingNumbers'].apply(lambda x: len(x) > 1)]
data

,level_0,row_id,name,address,description,pricerange,founder,owns,page_url,addressregion,streetaddress,addresscountry,telephone_address,addresslocality,postalcode,telephone,geo,sameas,latitude,longitude,url,aggregaterating,image,openinghours,review,email,contactpoint,reviews,employees,faxnumber,about,citystatezip,openinghoursspecification,areaserved,location,paymentaccepted,currenciesaccepted,legalname,photos,photo,logo,alternatename,brand,department,makesoffer,map,award,hasmap,isicv4,foundingdate,parentorganization,additionalproperty,city,state,county,employeeid,firstname,fullname,licensedata,nmlsid,officephone,suffdataforcta,numberofemployees,contactoption,ratingvalue,ratingcount,bestrating,worstrating,\u201caddresslocality\u201d,\u201cpostalcode\u201d,hasoffercatalog,itemreviewed,fax,postcode,startdate,enddate,amenityfeature,articlebody,naics,additionaltype,employee,whatsapp,datemodified,offers,category,community-number,knowslanguage,headline,author,datepublished,publisher,keywords,contacttype,postofficeboxnumber,vatid,member,additional_phone,breadcrumb,memberof,mainentityofpage,mainentity,interactioncount,slogan,elevation,distance,mobilebusiness,locationmobile,disambiguatingdescription,servicetype,potentialaction,servicearea,branchof,country,maximumattendeecapacity,knowsabout,containedinplace,id,isaccessibleforfree,branchcode,issimilarto,dateupdated,jobtitle,publicaccess,taxid,duns,hasmenu,identifier,leicode,servescuisine,mail,adresse,nom,validation%20de%20l'avis,gu00e9olocalisation,tu00e9lu00e9phone,inlanguage,reviewcount,reviewbody,caption,servicephone,maddress,smokingallowed,mobile,datecreated,alternativeheadline,events,contactpoints,price,itemlistelement,pricecurrency,availability,businessfunction,tollfree,reviewrating,clearfix,numberofrooms,eht,thumbnailurl,streetaddress2,website,blogpost,ispartof,event,webaddress,opens,closes,sku,\u201caddressregion\u201d,interactionstatistic,circle,title,listingemail,height,width,containsplace,maincontentofpage,hascredential,comment,georadius,place_location,place_id,place_name,dateposted,occupationalcategory,end_date,post_id,start_date,mpn,zipcode,productid,addressstate,specialopeninghoursspecification,suborganization,awards,founders,dealerzipcode,maplink,numberofitems,productsupported,foundinglocation,speakable,tourbookingpage,phone,itemprop=,"span>,",<span,participatesinmedicare,header-title-text,text-muted,containedin,primaryimageofpage,availablelanguage,userinteractioncount,acceptedpaymentmethod,hasmerchantreturnpolicy,postaladdress,address4,acceptsreservations,haspos,availableservice,position,type,virtualcategory,audience,restaurant,golfcourse,addressname,receivinghours,namecompany,provider,hascourseinstance,serialnumber,phonenumber,\u201daddresslocality\u201d,text,target,pickupinfo,\u201daddressregion\u201d,isrelatedto,telephone_,phone_object,E.164 format,telephoneNorm,indexValue,MatchingNumbers
91,93,102,Motomachi Restaurant Dubai,"{'addressregion': 'Dubai', 'addresslocality': ...","Motomachi Restaurant is a unique concept, whic...",NaN,NaN,NaN,https://sushipointer.com/places/united-arab-em...,Dubai,"Festival City Mall, Festival blvd, Al Rigga",AE,NaN,Dubai,8845,+971 4 385 5598,"{'latitude': '25.2219439', 'longitude': '55.35...",NaN,25.221944,55.352513,{'name': 'Motomachi Restaurant Dubai'},NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN

## Additional Filtering by Geo Location

In [87]:
def calcDifference(pointOne, pointTwo):
    return geopy.distance.great_circle(pointOne, pointTwo).km

def calcDifferenceFromRow (row):
    indexValue = row['indexValue']
    indexPosition = (row[lat], row[lon])
    diffList = []
    for value in row['MatchingGeoPoints']:
        currRow = data.loc[data['indexValue'] == value]
        currIndex = currRow['indexValue'].values[0]
        if currIndex == indexValue:
            diffList.append(-1)
        else:            
            currPosition = (currRow[lat].values[0], currRow[lon].values[0])
            diffList.append(calcDifference(indexPosition, currPosition))
    return diffList


In [ ]:
def createKDTree(tupleArray):
    tree = KDTree(tupleArray)
    return tree

# Return all values that are in a specific proximity
def queryTree(tree, point, radius = 0.001):    
    point = [float(i) for i in point]
    idx = tree.query_ball_point(point, r=radius)
    return idx
    #idx = tree.query(point, k=neighbours)
    #return idx[1]

# convert to tuples and from string to float
lonArr = data[lon].to_numpy()
latArr = data[lat].to_numpy()
tupleArray = np.array((lonArr, latArr)).T.astype('float32')

data.reset_index(drop=True, inplace=True)
data['indexValue'] = data.index

# create new column with all matching points
tree = createKDTree(tupleArray)
idx = queryTree(tree, tupleArray[0])

# Search for the closest neighbour in all of the points
data['MatchingGeoPoints'] = data.apply(lambda row: queryTree(tree,[row[lon], row[lat]]), axis=1) 

# Keep those that have one or more matches withing the radius
data = data[data['MatchingGeoPoints'].apply(lambda x: len(x) > 1)]

# Calculate the difference in km between those
data['Difference'] = data.apply(lambda row: calcDifferenceFromRow(row), axis=1) 

In [ ]:
data.head(5)

In [ ]:
len(data)

In [ ]:
data.loc[data['indexValue'] == 101]

In [ ]:
data.loc[data['indexValue'] == 3]

In [ ]:
data.loc[data['indexValue'] == 8]

In [ ]:
data.loc[data['indexValue'] == 95]

In [ ]:
data.to_csv('MatchingFile.csv')